In [ ]:
from src.data_management.dataset import CxrDatasetVer1, DatasetSplit
train_ds = CxrDatasetVer1(
    metadata_path='data/metadata/mimic-cxr-2.0.0-metadata.csv',
    split_path='data/metadata/mimic-cxr-2.0.0-split.csv',
    report_dir='data/mimic-cxr-reports',
    img_dir='data/cxr_dataset',
    split=DatasetSplit.TRAIN
)

val_ds = CxrDatasetVer1(
    metadata_path='data/metadata/mimic-cxr-2.0.0-metadata.csv',
    split_path='data/metadata/mimic-cxr-2.0.0-split.csv',
    report_dir='data/mimic-cxr-reports',
    img_dir='data/cxr_dataset',
    split=DatasetSplit.VAL
)

In [ ]:
from tqdm import tqdm


train_loader = torch.utils.data.DataLoader(
    train_ds, batch_size=CFG.batch_size, shuffle=True, num_workers=CFG.num_workers
)
val_loader = torch.utils.data.DataLoader(
    val_ds, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers
)

loss_meter = AvgMeter()
tqdm_object = tqdm(train_loader, total=len(train_loader))
model = CxrVQA().to("cuda")
optimizer = torch.optim.AdamW(
    model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay
)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=CFG.patience, factor=CFG.factor, min_lr=CFG.min_lr
)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

def valid_epoch(model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        imgs = batch[0].to("cuda")
        texts = batch[1]
        loss = model(imgs, texts)

        count = len(batch[0])
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter

best_loss = float('inf')
for epoch in range(CFG.epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()

    for batch in tqdm_object:
        imgs = batch[0].to("cuda")
        texts = batch[1]
        loss = model(imgs, texts)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step(loss)

        count = len(batch[0])
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))

    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, val_loader)
    
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), "best.pt")
        print("Saved Best Model!")

In [ ]:
for epoch in range(CFG.epochs):
    print(f"Epoch: {epoch + 1}")
    model.train()

    for batch in tqdm_object:
        imgs = batch[0].to("cuda")
        texts = batch[1]
        loss = model(imgs, texts)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step(loss)

        count = len(batch[0])
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))

    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, val_loader)
    
    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        torch.save(model.state_dict(), "best.pt")
        print("Saved Best Model!")

In [ ]:
learning_rate = get_lr(optimizer)

In [ ]:
dir(test.image_encoder)

In [ ]:
test = CxrVQA()
test(img.unsqueeze(0), [text])

In [ ]:
test = CxrResNet()

In [ ]:
img1 = img.unsqueeze(0)
img1.shape

In [ ]:
feats = test(img1)

In [ ]:
a = test.encoder.encode(img1)
a.shape

In [ ]:
# pool tensors torch.Size([1, 512, 3, 3]) into torch.Size([1, 512])
torch.nn.AdaptiveMaxPool2d((1, 1))(a).squeeze((2, 3)).shape

In [ ]:
import inspect
inspect.signature(test.encoder.features).parameters